# Intersectar buffers con radios y fracciones censales

Los datos del Censo Nacional 2010 están publicados en forma de radios y fracciones censales. Cada radio o fracción censal puede caer dentro de muchos buffers a la vez. Para componer indicadores publicados a nivel de radio censal en indicadores por buffer, cada buffer tiene que "saber" qué radios y fracciones censales tiene dentro suyo y en qué porcentaje de superficie.

El supuesto a utilizar a la hora de componer indicadores agregados será el de que la población (tanto en cantidad como en características) se distribuye homogéneamente dentro del radio o fracción. Posteriormente se podrá comparar el resultado entre la composición por radio y por fracción, para chequear si la utilización de la menor unidad de desagregación (el radio) representa una mejora estadísticamente significativa respecto de la utilización de fracciones (que son unidades más grandes, donde uno debería pensar que el supuesto de homogeneidad es más peligroso).

In [6]:
from scripts import calculate_weights
from scripts import geo_utils
import shapefile

In [7]:
fracciones_dir = "shp/divisiones/fracciones_caba_censo_2010"
fracciones_path = "shp/divisiones/fracciones_caba_censo_2010/fracciones_censo_2010"
subte_buffer500_path = "shp/transporte/buffers/estaciones_de_subte-buffer500/estaciones_de_subte-buffer500"

In [8]:
# para hacer pruebas
sf_fracciones = shapefile.Reader(fracciones_path)
fracciones = geo_utils.get_shapely_shapes(sf_fracciones)

sf_subte500 = shapefile.Reader(subte_buffer500_path)
estaciones = geo_utils.get_shapely_shapes(sf_subte500)

In [9]:
# para calcular los % de intersección de todos los buffers
# esto puede tomar bastante tiempo si se recalculan todos... (varias horas)
calculate_weights.main(recalculate=False)

Calculating intersections between fracciones_caba_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer1000
Calculating intersections between radios_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer1000
Calculating intersections between fracciones_caba_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer1500
Calculating intersections between radios_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer1500
Calculating intersections between fracciones_caba_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer2000
Calculating intersections between radios_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer2000
Calculating intersections between fracciones_caba_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer500
Calculating intersections between radios_censo_2010 and accesibilidad_en_estaciones_de_subte-buffer500
Calculating intersections between fracciones_caba_censo_2010 and estaciones_de_ferrocarril-buffer1000
Calculating intersections betwee